In [1]:
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from scipy import sparse, spatial
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

In [2]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)

print('gen_history_user_ques_accept...')

gen_history_user_ques_accept...


In [3]:
invite_info = pd.read_pickle(os.path.join(DATA_PATH, 'invite_info.pkl'))
invite_info_evaluate = pd.read_pickle(os.path.join(DATA_PATH, 'invite_info_evaluate.pkl'))
data = pd.read_pickle(os.path.join(DATA_PATH, 'invite_data.pkl'))

In [4]:
data_s = pd.concat([invite_info, invite_info_evaluate])
data = pd.concat([invite_info, invite_info_evaluate])
print(len(data), len(data_s))

10630845 10630845


In [5]:
# 问题曝光
tmp = data_s[['qid', 'itime']].groupby(['qid', 'itime']).size().reset_index()
tmp.columns = ['qid', 'itime', 'expo']
tmp = tmp.sort_values(by=['qid','itime'])
tmp['question_history_expo'] = tmp.groupby('qid')['expo'].cumsum() - tmp['expo']
tmp['question_time'] = tmp['qid'].astype(str) + '_' + tmp['itime'].astype(str)
data['question_time'] = data['qid'].astype(str) + '_' + data['itime'].astype(str)
data = data.merge(tmp[['question_time', 'question_history_expo']], 'left', 'question_time')

tmp = data_s[['qid', 'itime', 'label']].groupby(['qid', 'itime'])['label'].sum().reset_index()
tmp.columns = ['qid', 'itime', 'accept']
tmp = tmp.sort_values(by=['qid','itime'])
tmp['question_history_accept'] = tmp.groupby('qid')['accept'].cumsum() - tmp['accept']
tmp['question_time'] = tmp['qid'].astype(str) + '_' + tmp['itime'].astype(str)
data = data.merge(tmp[['question_time', 'question_history_accept']], 'left', 'question_time')

print(len(data))    

10630845


In [6]:
# 用户曝光
tmp = data_s[['uid', 'itime']].groupby(['uid', 'itime']).size().reset_index()
tmp.columns = ['uid', 'itime', 'expo']
tmp = tmp.sort_values(by=['uid','itime'])
tmp['user_history_expo'] = tmp.groupby('uid')['expo'].cumsum() - tmp['expo']
tmp['user_time'] = tmp['uid'].astype(str) + '_' + tmp['itime'].astype(str)
data['user_time'] = data['uid'].astype(str) + '_' + data['itime'].astype(str)
data = data.merge(tmp[['user_time', 'user_history_expo']], 'left', 'user_time')

tmp = data_s[['uid', 'itime', 'label']].groupby(['uid', 'itime'])['label'].sum().reset_index()
tmp.columns = ['uid', 'itime', 'accept']
tmp = tmp.sort_values(by=['uid','itime'])
tmp['user_history_accept'] = tmp.groupby('uid')['accept'].cumsum() - tmp['accept']
tmp['user_time'] = tmp['uid'].astype(str) + '_' + tmp['itime'].astype(str)
data = data.merge(tmp[['user_time', 'user_history_accept']], 'left', 'user_time')

print(len(data))

10630845


In [7]:
data['question_history_accept_rate'] = data['question_history_accept'] / data['question_history_expo']
data['user_history_accept_rate'] = data['user_history_accept'] / data['user_history_expo']

In [8]:
data[['question_history_accept', 'question_history_accept_rate', 'user_history_accept', 'user_history_accept_rate']].to_pickle(os.path.join(SAVE_PATH, 'history_user_ques_accept.pkl'))

In [9]:
toc = time.time()
print('Used time: %d' % int(toc-tic))

Used time: 194
